# Model 5
Hyperparameter tuning for XGBoost

## Restult
<code>
Best Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=1, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
</code >

## Future:
Data Manipulation along with hyperparamters tuning

In [5]:
# reading data from google drive 
# data paths whether notebook is run locally or google colab
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COMPETETION_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1"
    TRAIN_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/train.csv"
    TEST_DATA_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/data/test.csv"
    OUTPUT_PATH = "/content/drive/MyDrive/30-days-of-ml-competition1/output"
except:
    TRAIN_DATA_PATH = os.path.join("data", "train.csv")
    TEST_DATA_PATH = os.path.join("data", "test.csv")

print(f"Training Path {TRAIN_DATA_PATH}")
print(f"Testing Path {TEST_DATA_PATH}")



Training Path data/train.csv
Testing Path data/test.csv


In [6]:

# library imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# modeling
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# evaluation
from sklearn.metrics import mean_absolute_error

In [7]:
def read_organise_data(train=TRAIN_DATA_PATH):
    """read the data from a path and splitting to features and target

    Args:
        train (path, optional): The path of training data file to. Defaults to TRAIN_DATA_PATH.

    Returns:
        X, y: X for features and y for target
    """
    full_df = pd.read_csv(train, index_col="id")
    print(f"Shape of Dataset: {full_df.shape}")
    return full_df

df= read_organise_data()
X_test = pd.read_csv(TEST_DATA_PATH, index_col='id')
print(f"Shape of Test set {X_test.shape}")

Shape of Dataset: (300000, 25)
Shape of Test set (200000, 24)


## Understanding Data

In [6]:
df.describe()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,0.527335,0.460926,0.490498,0.496689,0.491654,0.510526,0.467476,0.537119,0.498456,0.474872,0.474492,0.473216,0.494561,0.508273,8.241979
std,0.230599,0.214003,0.253346,0.219199,0.240074,0.228232,0.210331,0.218140,0.239920,0.218007,0.255949,0.222022,0.247292,0.222950,0.746555
min,-0.118039,-0.069309,-0.056104,0.130676,0.255908,0.045915,-0.224689,0.203763,-0.260275,0.117896,0.048732,0.052608,-0.074208,0.151050,0.140329
25%,0.405965,0.310494,0.300604,0.329783,0.284188,0.354141,0.342873,0.355825,0.332486,0.306874,0.276017,0.308151,0.289074,0.300669,7.742071
50%,0.497053,0.427903,0.502462,0.465026,0.390470,0.488865,0.429383,0.504661,0.439151,0.434620,0.459975,0.433812,0.422887,0.472400,8.191373
75%,0.668060,0.615113,0.647512,0.664451,0.696599,0.669625,0.573383,0.703441,0.606056,0.614333,0.691579,0.642057,0.714502,0.758447,8.728634
max,1.058443,0.887253,1.034704,1.039560,1.055424,1.067649,1.111552,1.032837,1.040229,0.982922,1.055960,1.071444,0.975035,0.905992,10.411992


In [8]:
# checking for null values in the full dataset
df.isnull().any().sum() # no null value in dataset

0

In [8]:
y = df.target.copy()
X = df.drop('target', axis=1).copy()

In [9]:
# splitting the data
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2,
                                                      random_state= 1)

In [10]:
# categorical columns start with cat
categorical_cols = [cname for cname in X.columns if 'cat' in cname]
# numerical columns starts with cont
numerical_cols = [cname for cname in X.columns if 'cont' in cname]

In [11]:

# checking the cardinality of the categorical columns
for cname in categorical_cols:
    num_unique = X[cname].nunique()
    print(f"{cname} has {num_unique}")
    if num_unique > 10:
        print(f"\t{cname} has a high cardinality")

# eventhough cat9 col has more than 10 unique values, we will still use the OnehotEncoder 

cat0 has 2
cat1 has 2
cat2 has 2
cat3 has 4
cat4 has 4
cat5 has 4
cat6 has 8
cat7 has 8
cat8 has 7
cat9 has 15
	cat9 has a high cardinality


In [12]:

# numerical and categorical transformations
num_transformer = StandardScaler()
cat_transformer = OrdinalEncoder()
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, numerical_cols),
    ('cat', cat_transformer, categorical_cols),
])

In [13]:
def get_score(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    my_pipeline = Pipeline(steps=[
                                  ('preprocessor', preprocessor),
                                  ('model', model)
    ])
    my_pipeline.fit(X_t, y_t)
    preds = my_pipeline.predict(X_v)
    return mean_absolute_error(y_v, preds)
    

In [14]:
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=1)
get_score(xgb_model)


0.5740460076081315

In [32]:
X

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,1.0,1.0,0.0,0.0,1.0,3.0,0.0,4.0,0.0,8.0,...,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404
499996,0.0,1.0,0.0,2.0,1.0,1.0,0.0,4.0,4.0,5.0,...,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611
499997,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,6.0,5.0,...,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732


In [15]:
# encoder
ordinal_encode = OrdinalEncoder()
X[categorical_cols] = ordinal_encode.fit_transform(X[categorical_cols])
X

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
2,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
3,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
4,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
6,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,1.0,1.0,0.0,0.0,1.0,3.0,0.0,4.0,0.0,8.0,...,0.307883,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404
499996,0.0,1.0,0.0,2.0,1.0,1.0,0.0,4.0,4.0,5.0,...,0.736713,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611
499997,1.0,1.0,0.0,2.0,1.0,2.0,0.0,4.0,6.0,5.0,...,0.277074,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732


In [17]:
# parameters to search 
params = { 'max_depth': [3,6,10],
           'learning_rate': [0.01, 0.05],
           'n_estimators': [500, 1000, 1250, 1500]}

In [26]:
def param_tuning(params):

        model = XGBRegressor(n_estimators=1000, max_depth=3, 
                        learning_rate=0.01, objective='reg:squarederror', random_state=1)
        grid_search = GridSearchCV(estimator=model, 
                        param_grid=params,
                        scoring='neg_mean_absolute_error', 
                        verbose=3)
        grid_search.fit(X, y)
        print("Best parameters:", grid_search.best_params_)
        print("Lowest MAE: ", (-grid_search.best_score_))
        return grid_search

In [27]:
grid_search_results = param_tuning(params)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] learning_rate=0.01, max_depth=3, n_estimators=500 ...............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.01, max_depth=3, n_estimators=500, score=-0.580, total= 1.5min
[CV] learning_rate=0.01, max_depth=3, n_estimators=500 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=500, score=-0.585, total= 1.5min
[CV] learning_rate=0.01, max_depth=3, n_estimators=500 ...............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=3, n_estimators=500, score=-0.582, total= 1.6min
[CV] learning_rate=0.01, max_depth=3, n_estimators=500 ...............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=500, score=-0.583, total= 1.4min
[CV] learning_rate=0.01, max_depth=3, n_estimators=500 ...............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=500, score=-0.583, total= 1.4min
[CV] learning_rate=0.01, max_depth=3, n_estimators=1000 ..............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=1000, score=-0.577, total= 2.9min
[CV] learning_rate=0.01, max_depth=3, n_estimators=1000 ..............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=1000, score=-0.583, total= 2.9min
[CV] learning_rate=0.01, max_depth=3, n_estimators=1000 ..............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=1000, score=-0.579, total= 2.9min
[CV] learning_rate=0.01, max_depth=3, n_estimators=1000 ..............
[CV]  learning_rate=0.01, max_depth=3, n_estimators=1000, sco

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 727.2min finished


Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 1500}
Lowest MAE:  0.5726513246233991


In [28]:
grid_search_results

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=0.01, max_delta_step=None,
                                    max_depth=3, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estima...
                                    random_state=1, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),

In [34]:
print(f"Best Model {grid_search_results.best_estimator_}")
best_model =  grid_search_results.best_estimator_

Best Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=1, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)


In [35]:
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])

In [36]:
best_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10', 'cont11',
                                                   'cont12', 'cont13']),
                                                 ('cat',
                      

In [38]:
prediction_valid = best_pipeline.predict(X_valid)
print(mean_absolute_error(y_valid, prediction_valid))

0.5703273420755884


In [39]:
predictions = best_pipeline.predict(X_test)

In [40]:
OUTPUT_PATH = 'output'
def output_submission(prediction, file_name):
    """creating a kaggle submission file

    Args:
        prediction (array): an array of predictions of the test dataset
        file_name (string): a string for the name without the extension
    """
    my_submission = pd.DataFrame({'target': predictions},
                                 index=X_test.index)
    #my_submission.set_index('id')
    file_path = os.path.join(OUTPUT_PATH,file_name)
    my_submission.to_csv(f'{file_path}.csv')
    print(f'A submission file has been made at {file_path}')

In [41]:
output_submission(predictions, "Submission10")

A submission file has been made at output/Submission10


In [42]:
df

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,B,B,B,C,B,B,A,E,C,N,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
2,B,B,A,A,B,D,A,F,A,O,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
3,A,A,A,C,B,D,A,D,A,F,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
4,B,B,A,C,B,D,A,E,C,K,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
6,A,A,A,C,B,D,A,E,A,N,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,B,B,A,A,B,D,A,E,A,I,...,0.769792,0.450538,0.934360,1.005077,0.853726,0.422541,1.063463,0.697685,0.506404,7.945605
499996,A,B,A,C,B,B,A,E,E,F,...,0.528056,0.508502,0.358247,0.257825,0.433525,0.301015,0.268447,0.577055,0.823611,7.326118
499997,B,B,A,C,B,C,A,E,G,F,...,0.688747,0.372425,0.364936,0.383224,0.551825,0.661007,0.629606,0.714139,0.245732,8.706755
